# Movie Genie: Plan the perfect trip to the movies with just one click! 
By Kathleen, Rachel, Stephanie, and Zeeshan

Below you will find the backend code that we used to build our website Movie Genie.  Movie Genie asks the user a few questions about their day, analyzes their answers to predict their mood, and then displays showtimes at the nearest theaters for the movies that best fit their mood.  Having all results displayed in one place means less clicking around from website to website trying to find the best movie/showtime for you, and the sentiment analysis helps you decide what movie you should see.  MoviePass has made going to the movies trendy again, and with so many options out there (especially in NYC), we want to make the moviegoing process as easy as possible.

### Preparation

In [1]:
import pandas as pd
import requests
from datetime import datetime
from pytz import timezone
import json

est = timezone('US/Eastern')

### Showtimes of a selected genre in a specific zipcode

#### Get all cinemas in NYC 

To get all cinemas in NYC listed in the ishowtimes API, we make a call to the ishowtimes API using the city_id parameter for New York City to get an array containing all the cinemas in New York City.

In [2]:
ishowtimes_url = "https://api.internationalshowtimes.com/v4/cinemas"
parameters = {'city_id': '2215'}
headers={"X-API-Key": "pAnHCJdI8kzQKglyyhtJoUvV2rGnWDOR"}

NYCcinemas = (requests.get(ishowtimes_url, params=parameters, headers=headers).json())['cinemas']
NYCcinemas


[{'booking_type': 'external',
  'chain_id': '26',
  'id': '41514',
  'location': {'address': {'city': 'New York',
    'country': 'United States',
    'country_code': 'US',
    'display_text': '234 W 42nd St, New York, NY 10036, United States',
    'house': '234',
    'state': 'New York',
    'state_abbr': 'NY',
    'street': 'West 42nd Street',
    'zipcode': '10036'},
   'lat': 40.7567668,
   'lon': -73.9892626},
  'name': 'AMC Empire 25',
  'slug': 'amc-empire-25-new-york',
  'telephone': '(212) 398-2597',
  'website': 'https://www.amctheatres.com/movie-theatres/new-york-city/amc-empire-25'},
 {'booking_type': 'external',
  'chain_id': '27',
  'id': '41523',
  'location': {'address': {'city': 'New York',
    'country': 'United States',
    'country_code': 'US',
    'display_text': '850 Broadway, New York, NY 10003, United States',
    'house': '850',
    'state': 'New York',
    'state_abbr': 'NY',
    'street': 'Broadway',
    'zipcode': '10003'},
   'lat': 40.7340673,
   'lon': -73

In [3]:
# ishowtimes_url = "https://api.internationalshowtimes.com/v4/cinemas"
# parameters = {'country': 'US'}
# headers={"X-API-Key": "pAnHCJdI8kzQKglyyhtJoUvV2rGnWDOR"}

# UScinemas = (requests.get(ishowtimes_url, params=parameters, headers=headers).json())['cinemas']

# NYCcinemas = []

# for cinema in UScinemas:
#     if cinema['location']['address']['city'] == 'New York':
#         NYCcinemas.append(cinema)
        
# df2 = pd.DataFrame(NYCcinemas)
# df2

In [4]:
for cinema in NYCcinemas:
    if cinema['location']['address']['zipcode'] == '10034':
        print(cinema['name'])

#### Get all movie ids of 16 important genres
No genre 9: there are no movies under the genre "History" (genre id 9) out right now, so we left it out to avoid the errors it was causing.  Realistically, since movies are typically listed as multiple genres, any movie in the future classified as History will likely be also captured by another genre here and represented in our program (e.g. a Historical Drama, Documentary, War, etc.) 

In [5]:
genres = [
    {"id": "0", "title": "Action"},
    {"id": "1", "title": "Adventure"},
    {"id": "2", "title": "Animation"},
    {"id": "3", "title": "Comedy"},
    {"id": "4", "title": "Crime"},
    {"id": "5", "title": "Documentary"},
    {"id": "6", "title": "Drama"},
    {"id": "7", "title": "Family"},
    {"id": "8", "title": "Foreign"},
    {"id": "10", "title": "Horror"},
    {"id": "11", "title": "Mystery"},
    {"id": "12", "title": "Romance"},
    {"id": "13", "title": "Science Fiction"},
    {"id": "14", "title": "Thriller"},
    {"id": "15", "title": "War"},
    {"id": "16", "title": "Western"}
]

genres_ids = []

Next we have to call the ishowtimes API to access the movies by genre.  Unfortunately, because of the way the iShowtimes API is set up, we have to run separate calls for each genre which makes the program run rather slowly.  If you call the API and request all movies in a certain country or city, the results do not include the movie's genre and thus we couldn't simply make one call to gather the movies before sorting by genre.

In [6]:
# Genre 0
ishowtimes_url = "https://api.internationalshowtimes.com/v4/movies/"
parameters = {'country': 'US','genre_ids': '0'}
headers={"X-API-Key": "pAnHCJdI8kzQKglyyhtJoUvV2rGnWDOR"}
genre0 = requests.get(ishowtimes_url, params=parameters, headers=headers).json()['movies']

genre0_ids = [movie['id'] for movie in genre0]
genres_ids.append(genre0_ids)

In [7]:
# Genre 1
ishowtimes_url = "https://api.internationalshowtimes.com/v4/movies/"
parameters = {'country': 'US', 'genre_ids': '1'}
headers={"X-API-Key": "pAnHCJdI8kzQKglyyhtJoUvV2rGnWDOR"}
genre1 = requests.get(ishowtimes_url, params=parameters, headers=headers).json()['movies']

genre1_ids = [movie['id'] for movie in genre1]
genres_ids.append(genre1_ids)

In [8]:
# Genre 2
ishowtimes_url = "https://api.internationalshowtimes.com/v4/movies/"
parameters = {'country': 'US','genre_ids': '2'}
headers={"X-API-Key": "pAnHCJdI8kzQKglyyhtJoUvV2rGnWDOR"}
genre2 = requests.get(ishowtimes_url, params=parameters, headers=headers).json()['movies']

genre2_ids = [movie['id'] for movie in genre2]
genres_ids.append(genre2_ids)

In [9]:
# Genre 3
ishowtimes_url = "https://api.internationalshowtimes.com/v4/movies/"
parameters = {'country': 'US','genre_ids': '3'}
headers={"X-API-Key": "pAnHCJdI8kzQKglyyhtJoUvV2rGnWDOR"}
genre3 = requests.get(ishowtimes_url, params=parameters, headers=headers).json()['movies']

genre3_ids = [movie['id'] for movie in genre3]
genres_ids.append(genre3_ids)

In [10]:
# Genre 4
ishowtimes_url = "https://api.internationalshowtimes.com/v4/movies/"
parameters = {'country': 'US','genre_ids': '4'}
headers={"X-API-Key": "pAnHCJdI8kzQKglyyhtJoUvV2rGnWDOR"}
genre4 = requests.get(ishowtimes_url, params=parameters, headers=headers).json()['movies']

genre4_ids = [movie['id'] for movie in genre4]
genres_ids.append(genre4_ids)

In [11]:
# Genre 5
ishowtimes_url = "https://api.internationalshowtimes.com/v4/movies/"
parameters = {'country': 'US','genre_ids': '5'}
headers={"X-API-Key": "pAnHCJdI8kzQKglyyhtJoUvV2rGnWDOR"}
genre5 = requests.get(ishowtimes_url, params=parameters, headers=headers).json()['movies']

genre5_ids = [movie['id'] for movie in genre5]
genres_ids.append(genre5_ids)

In [12]:
# Genre 6
ishowtimes_url = "https://api.internationalshowtimes.com/v4/movies/"
parameters = {'country': 'US','genre_ids': '6'}
headers={"X-API-Key": "pAnHCJdI8kzQKglyyhtJoUvV2rGnWDOR"}
genre6 = requests.get(ishowtimes_url, params=parameters, headers=headers).json()['movies']

genre6_ids = [movie['id'] for movie in genre6]
genres_ids.append(genre6_ids)

In [13]:
# Genre 7
ishowtimes_url = "https://api.internationalshowtimes.com/v4/movies/"
parameters = {'country': 'US','genre_ids': '7'}
headers={"X-API-Key": "pAnHCJdI8kzQKglyyhtJoUvV2rGnWDOR"}
genre7 = requests.get(ishowtimes_url, params=parameters, headers=headers).json()['movies']

genre7_ids = [movie['id'] for movie in genre7]
genres_ids.append(genre7_ids)

In [14]:
# Genre 8
ishowtimes_url = "https://api.internationalshowtimes.com/v4/movies/"
parameters = {'country': 'US','genre_ids': '8'}
headers={"X-API-Key": "pAnHCJdI8kzQKglyyhtJoUvV2rGnWDOR"}
genre8 = requests.get(ishowtimes_url, params=parameters, headers=headers).json()['movies']

genre8_ids = [movie['id'] for movie in genre8]
genres_ids.append(genre8_ids)

In [15]:
# Genre 10
ishowtimes_url = "https://api.internationalshowtimes.com/v4/movies/"
parameters = {'country': 'US','genre_ids': '10'}
headers={"X-API-Key": "pAnHCJdI8kzQKglyyhtJoUvV2rGnWDOR"}
genre10 = requests.get(ishowtimes_url, params=parameters, headers=headers).json()['movies']

genre10_ids = [movie['id'] for movie in genre10]
genres_ids.append(genre10_ids)

In [16]:
# Genre 11
ishowtimes_url = "https://api.internationalshowtimes.com/v4/movies/"
parameters = {'country': 'US','genre_ids': '11'}
headers={"X-API-Key": "pAnHCJdI8kzQKglyyhtJoUvV2rGnWDOR"}
genre11 = requests.get(ishowtimes_url, params=parameters, headers=headers).json()['movies']

genre11_ids = [movie['id'] for movie in genre11]
genres_ids.append(genre11_ids)

In [17]:
# Genre 12
ishowtimes_url = "https://api.internationalshowtimes.com/v4/movies/"
parameters = {'country': 'US','genre_ids': '12'}
headers={"X-API-Key": "pAnHCJdI8kzQKglyyhtJoUvV2rGnWDOR"}
genre12 = requests.get(ishowtimes_url, params=parameters, headers=headers).json()['movies']

genre12_ids = [movie['id'] for movie in genre12]
genres_ids.append(genre12_ids)

In [18]:
# Genre 13
ishowtimes_url = "https://api.internationalshowtimes.com/v4/movies/"
parameters = {'country': 'US','genre_ids': '13'}
headers={"X-API-Key": "pAnHCJdI8kzQKglyyhtJoUvV2rGnWDOR"}
genre13 = requests.get(ishowtimes_url, params=parameters, headers=headers).json()['movies']

genre13_ids = [movie['id'] for movie in genre13]
genres_ids.append(genre13_ids)

In [19]:
# Genre 14
ishowtimes_url = "https://api.internationalshowtimes.com/v4/movies/"
parameters = {'country': 'US','genre_ids': '14'}
headers={"X-API-Key": "pAnHCJdI8kzQKglyyhtJoUvV2rGnWDOR"}
genre14 = requests.get(ishowtimes_url, params=parameters, headers=headers).json()['movies']

genre14_ids = [movie['id'] for movie in genre14]
genres_ids.append(genre14_ids)

In [20]:
# Genre 15
ishowtimes_url = "https://api.internationalshowtimes.com/v4/movies/"
parameters = {'country': 'US','genre_ids': '15'}
headers={"X-API-Key": "pAnHCJdI8kzQKglyyhtJoUvV2rGnWDOR"}
genre15 = requests.get(ishowtimes_url, params=parameters, headers=headers).json()['movies']

genre15_ids = [movie['id'] for movie in genre15]
genres_ids.append(genre15_ids)

In [21]:
# Genre 16
ishowtimes_url = "https://api.internationalshowtimes.com/v4/movies/"
parameters = {'country': 'US','genre_ids': '16'}
headers={"X-API-Key": "pAnHCJdI8kzQKglyyhtJoUvV2rGnWDOR"}
genre16 = requests.get(ishowtimes_url, params=parameters, headers=headers).json()['movies']

genre16_ids = [movie['id'] for movie in genre16]
genres_ids.append(genre16_ids)

#### Get all movie information

In [22]:
ishowtimes_url = "https://api.internationalshowtimes.com/v4/movies/"
parameters = {'country': 'US'}
headers={"X-API-Key": "pAnHCJdI8kzQKglyyhtJoUvV2rGnWDOR"}
movies_ids_titles = requests.get(ishowtimes_url, params=parameters, headers=headers).json()['movies']
movies_ids_titles

[{'id': '30767',
  'poster_image_thumbnail': 'http://image.tmdb.org/t/p/w154/7WsyChQLEftFiDOVTGkv3hFpyyt.jpg',
  'slug': 'avengers-infinity-war',
  'title': 'Avengers: Infinity War'},
 {'id': '27220',
  'poster_image_thumbnail': 'http://image.tmdb.org/t/p/w154/zWoFYfjEArXKjTJX6akSveGYSs3.jpg',
  'slug': 'deadpool-2',
  'title': 'Deadpool 2'},
 {'id': '31829',
  'poster_image_thumbnail': 'http://image.tmdb.org/t/p/w154/4oD6VEccFkorEBTEDXtpLAaz0Rl.jpg',
  'slug': 'han-solo-a-star-wars-story',
  'title': 'Solo: A Star Wars Story'},
 {'id': '31465',
  'poster_image_thumbnail': 'http://image.tmdb.org/t/p/w154/A2S4p7mRt0iyClSLSldfjzdYSIp.jpg',
  'slug': 'life-of-the-party',
  'title': 'Life of the Party'},
 {'id': '30227',
  'poster_image_thumbnail': 'http://image.tmdb.org/t/p/w154/mrepRTUhNKU70PFf7LNQypbkH00.jpg',
  'slug': 'a-quiet-place',
  'title': 'A Quiet Place'},
 {'id': '40796',
  'poster_image_thumbnail': 'http://image.tmdb.org/t/p/w154/bKH7H4f5LPBQ5KOmHPcYtxCo7Od.jpg',
  'slug': 'b

#### Get all cinemas in a zipcode

In [23]:
def cinemas_in_zipcode(zipcode):
    cinemas_in_zipcode = []
    for cinema in NYCcinemas:
        if cinema['location']['address']['zipcode'] == zipcode:
            cinemas_in_zipcode.append(cinema)
    return cinemas_in_zipcode

#### Get showtimes of all genres of all cinemas in a zipcode 
Only showtimes of current day and after current time

In [24]:
def genres_showtimes_in_zipcode(zipcode):
    
    # Get all showtimes of all cinemas in a specific zipcode 
    showtimes_in_zipcode = []
    
    for cinema in cinemas_in_zipcode(zipcode):        
        cinema_id = cinema['id']
        ishowtimes_url = "https://api.internationalshowtimes.com/v4/showtimes/?cinema_id={}".format(cinema_id)
        headers = {"X-API-Key": "pAnHCJdI8kzQKglyyhtJoUvV2rGnWDOR"}
        cinema_showtimes = requests.get(ishowtimes_url,headers = headers).json()['showtimes']

        showtimes_in_zipcode.append(cinema_showtimes)
            
    # Get showtimes (only current day after current time) for all movies of each genre
    genres_showtimes_in_zipcode = [] 
    
    for genre in genres_ids:
        genre_showtimes_in_zipcode = []
        
        for cinema in showtimes_in_zipcode:
            for movie in cinema:
                criterion1 = movie['movie_id'] in genre
                # API gives the local time with a time zone offset in hours and minutes
                # Positive offset: local time is ahead of UTC; Negative offset: local time is behind UTC
                # UTC (Coordinated Universal Time) is 4 hours ahead of EDT (Eastern Daylight Time)
                # eg: 2018-01-01T15:30:00-04:00 
                # 2018-01-01T15:30:00 in EDT
                # 2018-01-01T19:30:00 in UTC
                criterion2 = movie['start_at'].startswith(datetime.now(tz=est).strftime('%Y-%m-%d'))                
                criterion3 = est.localize(datetime.strptime(movie['start_at'],'%Y-%m-%dT%H:%M:%S-04:00')) > datetime.now(tz=est)
                if criterion1 and criterion2 and criterion3:
                    genre_showtimes_in_zipcode.append(movie)
                    
        genres_showtimes_in_zipcode.append(genre_showtimes_in_zipcode)
        
    return genres_showtimes_in_zipcode   

#### Get showtimes of a specific genre

In [25]:
def genre_showtimes_selected(zipcode, genre_title):

    for genre in genres:
        if genre['title'] == genre_title:
            genre_loc = int(genre['id'])
            if genre_loc >= 10:
                genre_loc -= 1
                
    return genres_showtimes_in_zipcode(zipcode)[genre_loc]   

####  Final results

In [26]:
def results(zipcode, genre_title):
    results = []
    
    for showtime in genre_showtimes_selected(zipcode, genre_title):
        info = {}
        
        for movie in movies_ids_titles:
            if movie['id'] == showtime['movie_id']:
                info['Title'] = movie['title']
                info["Poster link"] = movie['poster_image_thumbnail']
                
        for cinema in cinemas_in_zipcode(zipcode):
            if cinema['id'] == showtime['cinema_id']:
                info['Cinema'] = cinema['name']
                info["Location"] = cinema['location']['address']['display_text']
                
        info["Start at"] = showtime['start_at'].replace("T", " ").replace("-04:00","")
        results.append(info)

    
    return results

### Sentiment Analysis

Now that we have functions to fetch movies and movie times based on genre and zipcode, we want to be able to determine what genre the user should watch.  To do this, we will ask them some questions and do a sentiment analysis of thier responses.   

In [27]:

def getSentiment(text):
    #Use IBM Watson API to perform a sentiment analysis on the text we will get from the user input
    endpoint = "https://gateway.watsonplatform.net/natural-language-understanding/api/v1/analyze"

   
    username = "25a70137-e1e3-4e34-8ca3-daffb770c96b"
    password = "4NrzHVNT7KBj"

    parameters = {
        'features': 'emotion,sentiment',
        'version' : '2017-03-16',
        'text': text,
        'language' : 'en',
    }

    resp = requests.get(endpoint, params=parameters, auth=(username, password))
    
    return resp.json()


In [28]:
print("Hi, I am looking forward to helping you choose a movie for tonight. Please answer each question I ask in one or two sentences.")

Hi, I am looking forward to helping you choose a movie for tonight. Please answer each question I ask in one or two sentences.


We want to ask users random questions and take their input to analyze their mood.

In [29]:
answer1 = input("What is the most exciting thing that happened to you today? ")

What is the most exciting thing that happened to you today? photos


In [30]:
answer2 = input("What are you looking forward to tonight? ")

What are you looking forward to tonight? drinks


In [31]:
answer3 = input("If you were granted one wish right now, what would you wish for? (don't ask for more wishes...) ")

If you were granted one wish right now, what would you wish for? (don't ask for more wishes...) nothing


In [32]:
answer4 = input("Describe the weather outside. ")

Describe the weather outside. stormy


In [33]:
answer5 = input("How are you feeling today? ")
print("Thanks! We're finding the perfect movie for your mood.")

How are you feeling today? good
Thanks! We're finding the perfect movie for your mood.


In [34]:
zipcode = input("Finally, what is your zipcode? ")

Finally, what is your zipcode? 10035


In [35]:
#Put all the answers into one string to be analyzed by the watson API
answers = answer1 + " " + answer2 + " " + answer3 + " " + answer4 + " " + answer5


Run the getSentiment function defined above to get the sentiment of the user responses.  Store the results

In [36]:
data = getSentiment(answers)
#Not necessary for the program, but print sentiment analysis results to see that it worked
data

{'emotion': {'document': {'emotion': {'anger': 0.058538,
    'disgust': 0.07813,
    'fear': 0.02014,
    'joy': 0.727802,
    'sadness': 0.093674}}},
 'language': 'en',
 'sentiment': {'document': {'label': 'neutral', 'score': 0.0}},
 'usage': {'features': 2, 'text_characters': 33, 'text_units': 1}}

Access just the "emotion" dictionary from the getSentiment output and store it 

In [37]:
emotions = data["emotion"]["document"]["emotion"]

Assign genres that correspond to each dominant emotion.
e.g. If anger has a higher percentage than the other 4 emotions, recommend Action, War, and Western

In [38]:
if emotions["anger"] > emotions["disgust"] and emotions["anger"] > emotions["fear"] and emotions["anger"] > emotions["joy"] and emotions["anger"] > emotions["sadness"]:
    #Action, War, and Western to act as an outlet for the anger
    genre_id = [0,15,16]

if emotions["disgust"] > emotions["anger"] and emotions["disgust"] > emotions["fear"] and emotions["disgust"] > emotions["joy"] and emotions["disgust"] > emotions["sadness"]: 
    #Crime, Documentary, and Romance to cater to different kinds of disgust the user may be feeling
    genre_id = [4,5,12]
  
    
if emotions["fear"] > emotions["anger"] and emotions["fear"] > emotions["disgust"] and emotions["fear"] > emotions["joy"] and emotions["fear"] > emotions["sadness"]:
    #Horror, Sci-Fi, and Thriller for the user already in a fearful mood
    genre_id = [10,13,14]


if emotions["joy"] > emotions["anger"] and emotions["joy"] > emotions["fear"] and emotions["joy"] > emotions["disgust"] and emotions["joy"] > emotions["sadness"]:
    #Animation, Comedy, and Family for the happy user 
    genre_id = [2,3,7]


else:
    #If the user is sad, they get a variety of options -- Advernture, Drama, Foreign, and Mystery 
    genre_id = [1,6,8,11]

Sort zipcodes by neighborhood, and find the movie times for all the theaters in the neighborhood (not just in the zipcode).  Some zipcodes don't have movie theaters in the ishowtames database, and we wanted to give the users more options. 

In [39]:
zipcodes = [
    ("10026", "10027", "10030", "10037", "10039"),
    ("10001", "10011", "10018", "10019", "10020", "10036"),
    ("10029", "10035"),
    ("10010", "10016", "10017", "10022"),
    ("10012", "10013", "10014"),
    ("10004", "10005", "10006", "10007", "10038", "10281"),
    ("10002", "10003", "10009"),
    ("10021", "10028", "10044", "10065", "10075", "10128"),
    ("10023", "10024", "10025"),
    ("10031", "10032", "10033", "10034", "10040")
]


Get the list of all the zipcodes in the neighborhood from the zipcodes dictionary based on the user's zipcode input.  Some neighborhoods don't have any theaters listed in the ishowtimes database, so we reassigned the zipcodes in those neighborhoods to the nearest neighborhood.

In [40]:
def getZipcodes(zipcode):
    if zipcode == '10013':
        zip_list = zipcodes[5]
    elif zipcode == '10012':
        zip_list = zipcodes[6]
    elif zipcode == '10029':
        zip_list = zipcodes[7]
    elif zipcode == '10035' or zipcode in zipcodes[9]:
        zip_list = zipcodes[0]
    elif zipcode == '10014':
        zip_list = zipcodes[1]
    else:    
        for neighborhood in zipcodes:
            for code in neighborhood:
                if code == zipcode:
                    zip_list = neighborhood
    return zip_list
                

In [41]:
#Store the zipcodes as their own list
zips = getZipcodes(zipcode)
zips

('10026', '10027', '10030', '10037', '10039')

Get the results for all movie times for the genres and zipcodes corresponding to the user's input.  Store them in an array.

In [42]:
all_results = []
for genre in genre_id:
    for code in zips:
        movie_results = results(code, genres[genre]["title"])
        for result in movie_results:
            #This takes care of the issue pointed out in the class presentation
            #Some movies (namely Isle of Dogs) were being double counted... 
            #because they fit more than one genre recommended for one sentiment
            #Now, there should be no duplicates becasue the result is only appended...
            #if it doesn't already exist in the all_results array
            if not result in all_results:
                all_results.append(result)
            
all_results

[]

To display the results in a way that's easier to read, we can create a pandas dataframe with the data. The output is the final results the user will receive.

In [44]:
movie_data = pd.DataFrame(all_results)
if len(movie_data) == 0:
    movies = "Sorry, there are no more showings tonight in your area to match your mood."
else:
    movies = movie_data.sort_values(by=['Start at'])

movies

'Sorry, there are no more showings tonight in your area to match your mood.'

# Going from back end to front end with Flask
Below you will find the code from our .py and .html notebooks that we used to create the front end aspect of this data product.  We took the code we used in our backend (presented above), and modified it slightly to better operate with Flask.  The main modification was using fewer functions to perform the same tasks.   

## questionnaire.py

```python
from flask import Flask, render_template, request
app = Flask(__name__)
import requests
import json
import pandas as pd
from datetime import datetime
from pytz import timezone
est = timezone('US/Eastern')

@app.route('/')
def student():
    return render_template('user_inputs.html')

@app.route('/result',methods = ['POST'])
def result():
    answer1 = request.form['Q1']
    answer2 = request.form['Q2']
    answer3 = request.form['Q3']
    answer4 = request.form['Q4']
    answer5 = request.form['Q5']
    zipcode = request.form['Q6']
    
    answers = answer1 + " " + answer2 + " " + answer3 + " " + answer4 + " " + answer5
    
    data = getSentiment(answers)
    
    emotions = data["emotion"]["document"]["emotion"]
    
    if emotions["anger"] > emotions["disgust"] and emotions["anger"] > emotions["fear"] and emotions["anger"] > emotions["joy"] and emotions["anger"] > emotions["sadness"]:
    #Action, War, and Western to act as an outlet for the anger
        genre_id = [0,15,16]

    if emotions["disgust"] > emotions["anger"] and emotions["disgust"] > emotions["fear"] and emotions["disgust"] > emotions["joy"] and emotions["disgust"] > emotions["sadness"]: 
    #Crime, Documentary, and Romance to cater to different kinds of disgust the user may be feeling
        genre_id = [4,5,12]
  
    
    if emotions["fear"] > emotions["anger"] and emotions["fear"] > emotions["disgust"] and emotions["fear"] > emotions["joy"] and emotions["fear"] > emotions["sadness"]:
    #Horror, Sci-Fi, and Thriller for the user already in a fearful mood
        genre_id = [10,13,14]


    if emotions["joy"] > emotions["anger"] and emotions["joy"] > emotions["fear"] and emotions["joy"] > emotions["disgust"] and emotions["joy"] > emotions["sadness"]:
    #Animation, Comedy, and Family for the happy user 
        genre_id = [2,3,7]


    else:
    #If the user is sad, they get a variety of options -- Advernture, Drama, Foreign, and Mystery 
        genre_id = [1,6,8,11]
    
    genres = [
        {"id": "0", "title": "Action"},
        {"id": "1", "title": "Adventure"},
        {"id": "2", "title": "Animation"},
        {"id": "3", "title": "Comedy"},
        {"id": "4", "title": "Crime"},
        {"id": "5", "title": "Documentary"},
        {"id": "6", "title": "Drama"},
        {"id": "7", "title": "Family"},
        {"id": "8", "title": "Foreign"},
        {"id": "9", "title": "History"},
        {"id": "10", "title": "Horror"},
        {"id": "11", "title": "Mystery"},
        {"id": "12", "title": "Romance"},
        {"id": "13", "title": "Science Fiction"},
        {"id": "14", "title": "Thriller"},
        {"id": "15", "title": "War"},
        {"id": "16", "title": "Western"}
        ]

                    
    ishowtimes_url = "https://api.internationalshowtimes.com/v4/cinemas"
    parameters = {'city_id': '2215'}
    headers={"X-API-Key": "pAnHCJdI8kzQKglyyhtJoUvV2rGnWDOR"}

    NYCcinemas = (requests.get(ishowtimes_url, params=parameters, headers=headers).json())['cinemas']
     


    genres_ids = []

    ishowtimes_url = "https://api.internationalshowtimes.com/v4/movies/"
    parameters = {'country': 'US','genre_ids': '0'}
    headers={"X-API-Key": "pAnHCJdI8kzQKglyyhtJoUvV2rGnWDOR"}
    genre0 = requests.get(ishowtimes_url, params=parameters, headers=headers).json()['movies']

    genre0_ids = [movie['id'] for movie in genre0]
    genres_ids.append(genre0_ids)

    ishowtimes_url = "https://api.internationalshowtimes.com/v4/movies/"
    parameters = {'country': 'US', 'genre_ids': '1'}
    headers={"X-API-Key": "pAnHCJdI8kzQKglyyhtJoUvV2rGnWDOR"}
    genre1 = requests.get(ishowtimes_url, params=parameters, headers=headers).json()['movies']

    genre1_ids = [movie['id'] for movie in genre1]
    genres_ids.append(genre1_ids)

    ishowtimes_url = "https://api.internationalshowtimes.com/v4/movies/"
    parameters = {'country': 'US','genre_ids': '2'}
    headers={"X-API-Key": "pAnHCJdI8kzQKglyyhtJoUvV2rGnWDOR"}
    genre2 = requests.get(ishowtimes_url, params=parameters, headers=headers).json()['movies']

    genre2_ids = [movie['id'] for movie in genre2]
    genres_ids.append(genre2_ids)

    ishowtimes_url = "https://api.internationalshowtimes.com/v4/movies/"
    parameters = {'country': 'US','genre_ids': '3'}
    headers={"X-API-Key": "pAnHCJdI8kzQKglyyhtJoUvV2rGnWDOR"}
    genre3 = requests.get(ishowtimes_url, params=parameters, headers=headers).json()['movies']

    genre3_ids = [movie['id'] for movie in genre3]
    genres_ids.append(genre3_ids)

    ishowtimes_url = "https://api.internationalshowtimes.com/v4/movies/"
    parameters = {'country': 'US','genre_ids': '4'}
    headers={"X-API-Key": "pAnHCJdI8kzQKglyyhtJoUvV2rGnWDOR"}
    genre4 = requests.get(ishowtimes_url, params=parameters, headers=headers).json()['movies']

    genre4_ids = [movie['id'] for movie in genre4]
    genres_ids.append(genre4_ids)

    ishowtimes_url = "https://api.internationalshowtimes.com/v4/movies/"
    parameters = {'country': 'US','genre_ids': '5'}
    headers={"X-API-Key": "pAnHCJdI8kzQKglyyhtJoUvV2rGnWDOR"}
    genre5 = requests.get(ishowtimes_url, params=parameters, headers=headers).json()['movies']

    genre5_ids = [movie['id'] for movie in genre5]
    genres_ids.append(genre5_ids)

    ishowtimes_url = "https://api.internationalshowtimes.com/v4/movies/"
    parameters = {'country': 'US','genre_ids': '6'}
    headers={"X-API-Key": "pAnHCJdI8kzQKglyyhtJoUvV2rGnWDOR"}
    genre6 = requests.get(ishowtimes_url, params=parameters, headers=headers).json()['movies']

    genre6_ids = [movie['id'] for movie in genre6]
    genres_ids.append(genre6_ids)

    ishowtimes_url = "https://api.internationalshowtimes.com/v4/movies/"
    parameters = {'country': 'US','genre_ids': '7'}
    headers={"X-API-Key": "pAnHCJdI8kzQKglyyhtJoUvV2rGnWDOR"}
    genre7 = requests.get(ishowtimes_url, params=parameters, headers=headers).json()['movies']

    genre7_ids = [movie['id'] for movie in genre7]
    genres_ids.append(genre7_ids)

    ishowtimes_url = "https://api.internationalshowtimes.com/v4/movies/"
    parameters = {'country': 'US','genre_ids': '8'}
    headers={"X-API-Key": "pAnHCJdI8kzQKglyyhtJoUvV2rGnWDOR"}
    genre8 = requests.get(ishowtimes_url, params=parameters, headers=headers).json()['movies']

    genre8_ids = [movie['id'] for movie in genre8]
    genres_ids.append(genre8_ids)

    ishowtimes_url = "https://api.internationalshowtimes.com/v4/movies/"
    parameters = {'country': 'US','genre_ids': '10'}
    headers={"X-API-Key": "pAnHCJdI8kzQKglyyhtJoUvV2rGnWDOR"}
    genre10 = requests.get(ishowtimes_url, params=parameters, headers=headers).json()['movies']

    genre10_ids = [movie['id'] for movie in genre10]
    genres_ids.append(genre10_ids)

    ishowtimes_url = "https://api.internationalshowtimes.com/v4/movies/"
    parameters = {'country': 'US','genre_ids': '11'}
    headers={"X-API-Key": "pAnHCJdI8kzQKglyyhtJoUvV2rGnWDOR"}
    genre11 = requests.get(ishowtimes_url, params=parameters, headers=headers).json()['movies']

    genre11_ids = [movie['id'] for movie in genre11]
    genres_ids.append(genre11_ids)

    ishowtimes_url = "https://api.internationalshowtimes.com/v4/movies/"
    parameters = {'country': 'US','genre_ids': '12'}
    headers={"X-API-Key": "pAnHCJdI8kzQKglyyhtJoUvV2rGnWDOR"}
    genre12 = requests.get(ishowtimes_url, params=parameters, headers=headers).json()['movies']

    genre12_ids = [movie['id'] for movie in genre12]
    genres_ids.append(genre12_ids)

    ishowtimes_url = "https://api.internationalshowtimes.com/v4/movies/"
    parameters = {'country': 'US','genre_ids': '13'}
    headers={"X-API-Key": "pAnHCJdI8kzQKglyyhtJoUvV2rGnWDOR"}
    genre13 = requests.get(ishowtimes_url, params=parameters, headers=headers).json()['movies']

    genre13_ids = [movie['id'] for movie in genre13]
    genres_ids.append(genre13_ids)

    ishowtimes_url = "https://api.internationalshowtimes.com/v4/movies/"
    parameters = {'country': 'US','genre_ids': '14'}
    headers={"X-API-Key": "pAnHCJdI8kzQKglyyhtJoUvV2rGnWDOR"}
    genre14 = requests.get(ishowtimes_url, params=parameters, headers=headers).json()['movies']

    genre14_ids = [movie['id'] for movie in genre14]
    genres_ids.append(genre14_ids)

    ishowtimes_url = "https://api.internationalshowtimes.com/v4/movies/"
    parameters = {'country': 'US','genre_ids': '15'}
    headers={"X-API-Key": "pAnHCJdI8kzQKglyyhtJoUvV2rGnWDOR"}
    genre15 = requests.get(ishowtimes_url, params=parameters, headers=headers).json()['movies']

    genre15_ids = [movie['id'] for movie in genre15]
    genres_ids.append(genre15_ids)

    ishowtimes_url = "https://api.internationalshowtimes.com/v4/movies/"
    parameters = {'country': 'US','genre_ids': '16'}
    headers={"X-API-Key": "pAnHCJdI8kzQKglyyhtJoUvV2rGnWDOR"}
    genre16 = requests.get(ishowtimes_url, params=parameters, headers=headers).json()['movies']

    genre16_ids = [movie['id'] for movie in genre16]
    genres_ids.append(genre16_ids)



    ishowtimes_url = "https://api.internationalshowtimes.com/v4/movies/"
    parameters = {'country': 'US'}
    headers={"X-API-Key": "pAnHCJdI8kzQKglyyhtJoUvV2rGnWDOR"}
    movies_ids_titles = requests.get(ishowtimes_url, params=parameters, headers=headers).json()['movies']

    zipcodes = [
    ("10026", "10027", "10030", "10037", "10039"),
    ("10001", "10011", "10018", "10019", "10020", "10036"),
    ("10029", "10035"),
    ("10010", "10016", "10017", "10022"),
    ("10012", "10013", "10014"),
    ("10004", "10005", "10006", "10007", "10038", "10280"),
    ("10002", "10003", "10009"),
    ("10021", "10028", "10044", "10065", "10075", "10128"),
    ("10023", "10024", "10025"),
    ("10031", "10032", "10033", "10034", "10040")
    ]
    
    if zipcode == '10013':
        zip_list = zipcodes[5]
    elif zipcode == '10012':
        zip_list = zipcodes[6]
    elif zipcode == '10029':
        zip_list = zipcodes[7]
    elif zipcode == '10035' or zip in zipcodes[9]:
        zip_list = zipcodes[0]
    elif zipcode == '10014':
        zip_list = zipcodes[1]
    else:    
        for neighborhood in zipcodes:
            for code in neighborhood:
                if code == zipcode:
                    zip_list = neighborhood
                
    results = []

    cinemas_in_neighborhood = []
    for cinema in NYCcinemas:
        for x in zip_list:
            if cinema['location']['address']['zipcode'] == x:
                cinemas_in_neighborhood.append(cinema)

    showtimes_in_neighborhood = []
    for cinema in cinemas_in_neighborhood:
        cinema_id = cinema['id']
        ishowtimes_url = "https://api.internationalshowtimes.com/v4/showtimes/?cinema_id={}".format(cinema_id)
        headers = {"X-API-Key": "pAnHCJdI8kzQKglyyhtJoUvV2rGnWDOR"}
        cinema_showtimes = requests.get(ishowtimes_url,headers = headers).json()['showtimes']
        showtimes_in_neighborhood.append(cinema_showtimes)


    genres_showtimes_in_neighborhood = [] 
    for genre in genres_ids:
        genre_showtimes_in_neighborhood = []

        for cinema in showtimes_in_neighborhood:
            for movie in cinema:
                criterion1 = movie['movie_id'] in genre
                criterion2 = movie['start_at'].startswith(datetime.now(tz=est).strftime('%Y-%m-%d'))                
                criterion3 = est.localize(datetime.strptime(movie['start_at'],'%Y-%m-%dT%H:%M:%S-04:00'))> datetime.now(tz=est)
                if criterion1 and criterion2 and criterion3:
                    genre_showtimes_in_neighborhood.append(movie)

        genres_showtimes_in_neighborhood.append(genre_showtimes_in_neighborhood)


    for genre in genre_id:
        if genre >= 10:
            genre -= 1
        for showtime in genres_showtimes_in_neighborhood[genre]:
            info = {}

            for movie in movies_ids_titles:
                if movie['id'] == showtime['movie_id']:
                    info['Title'] = movie['title']
                    info["Poster link"] = movie['poster_image_thumbnail']

            for cinema in cinemas_in_neighborhood:
                if cinema['id'] == showtime['cinema_id']:
                    info['Cinema'] = cinema['name']
                    info["Location"] = cinema['location']['address']['display_text']

            info["Start at"] = showtime['start_at'].replace("T", " ").replace("-04:00","")
            if not info in results:
                results.append(info)

    if len(results) > 0:
        results = pd.DataFrame(results).sort_values(by = 'Start at')
    else:
        results = pd.DataFrame(results) 
    

    return render_template("output.html", results = results)
    

def getSentiment(text):
    #We're using the IBM Watson API to perform a sentiment analysis
    endpoint = "https://gateway.watsonplatform.net/natural-language-understanding/api/v1/analyze"

   
    username = "25a70137-e1e3-4e34-8ca3-daffb770c96b"
    password = "4NrzHVNT7KBj"

    parameters = {
        #'features' : 'concepts,categories,emotion,entities,keywords,metadata,relations,semantic_roles,sentiment',
        'features': 'emotion,sentiment',
        'version' : '2017-03-16',
        'text': text,
        'language' : 'en',
        # url = url_to_analyze, this is an alternative to sending the text
    }

    resp = requests.get(endpoint, params=parameters, auth=(username, password))
    
    return resp.json()
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=True)
            
```  


## user_inputs.html
Found in the templates folder under PPDS_Final_Project (the title of our project folder in jupyter)

File 'user_inputs.html' in folder 'templates'

```html
<!DOCTYPE html>

<html>
 <table height="100%" width="100%" cellpadding="0" cellspacing="0" border="0">
   <tr>
     <td valign="top" align="left" background="https://images.unsplash.com/photo-1521021153866-f8089b0aa617?ixlib=rb-0.3.5&ixid=eyJhcHBfaWQiOjEyMDd9&s=94418b9bd265c9b30fd8c1f58dd9515c&auto=format&fit=crop&w=1350&q=80"> 
    <head>
    <h1 style="color: #FFFFFF">&nbsp;Welcome to Movie Genie</h1>
    <h3 style="color: #FFFFFF">&nbsp;&nbsp;Answer the following questions so we can help you plan your perfect night!</h3>
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" integrity="sha384-BVYiiSIFeK1dGmJRAkycuHAHRg32OmUcww7on3RYdg4Va+PmSTsz/K68vbdEjh4u" crossorigin="anonymous">
</head>
<body>
   
      <form action = "http://52.204.127.21:5000/result" method = "POST">
          <p style="color: #FFFFFF">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;What is the most interesting thing that has happened to you today?</p>
          <p>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<input type="text" name="Q1"></p>
          <p style="color: #FFFFFF">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;What are you looking forward to later?</p>
          <p>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<input type="text" name="Q2"></p>
          <p style="color: #FFFFFF">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;If you could be granted one wish, what would you wish for? </p>
          <p>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<input type="text" name="Q3"></p>
          <p style="color: #FFFFFF">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Describe the weather today.</p>
          <p>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<input type="text" name="Q4"></p>
          <p style="color: #FFFFFF">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;How are you feeling today?</p>
          <p>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<input type="text" name="Q5"></p>
          <p style="color: #FFFFFF">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Lastly, what is your zip code?</p>
          <p>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<input type="text" name="Q6"></p>
          <p>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<input type="submit">
          <p>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</p>
          <p>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</p>
          <p>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</p>
          <p>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</p>
          <p>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</p>
          <p>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</p>
          <p>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</p>
          <p>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</p>
          <p>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</p>
          <p>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</p>
          <p>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</p>
          
      </form>


</body>
</td>
   </tr>
 </table>
</html>
```
        

## output.html
This file formats the pandas dataframe displayed to the user when they get their results.

File 'output.html' in folder 'templates'

```html
<!DOCTYPE html>
<html>
<head>
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" integrity="sha384-BVYiiSIFeK1dGmJRAkycuHAHRg32OmUcww7on3RYdg4Va+PmSTsz/K68vbdEjh4u" crossorigin="anonymous">
</head>
<body> 
    <div class="container">
        <div class="panel panel-info">
            <div class="panel-heading">
                {% if results|length == 0 %}
                        <p>Sorry, there are no more shows today in your area to fit your mood.</p>
                {% else %}
                    <h1 class="panel-title">The perfect movies for your mood: </h1>
            </div>
            <table  class="table table-striped table-bordered table-hover">
                <thead>
                     <tr>    <!-- Construct table header -->
                            <th> Title </th>
                            <th> Start at </th>
                            <th> Cinema </th>
                            <th> Location </th>
                    </tr>
                </thead>
                <tbody>
                    
             
                    
                         {% for index, row in results.iterrows() %}
                        <tr>
                            <td>{{ row['Title'] }}</td>
                            <td>{{ row['Start at'] }}</td>
                            <td>{{ row['Cinema'] }}</td>
                            <td>{{ row['Location'] }}</td>
                        </tr>
                    {% endfor %}
                    
                    {% endif %}
                   
                     </tbody>
             </table>
        </div>
    </div>
</body>
</html>
```

Going forward, we would want to find a faster and easier to use API to make the user experience better,  and we would want a more detailed sentiment analysis to truly specialize the movie selections for your mood.  We think this could also be expanded to recommending certain restaurants or Airbnbs based on your mood or personality.  But for now, Movie Genie can be a helpful way for users to more easily plan their trip to the movies. 